# Title

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging
from collections.abc import *
from dataclasses import KW_ONLY, dataclass
from typing import Any, NamedTuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from pandas import DataFrame, Index, MultiIndex, Series

rng = np.random.default_rng()
np.set_printoptions(precision=4, floatmode="fixed", suppress=True)
logging.basicConfig(level=logging.INFO)

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from torch import Tensor
from torch import nan as NAN
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import Sampler as TorchSampler

from tsdm.datasets import KiwiDataset, TimeSeriesCollection
from tsdm.random.samplers import HierarchicalSampler, SlidingWindowSampler
from tsdm.tasks import TimeSeriesSampleGenerator, TimeSeriesTask
from tsdm.tasks.base import Batch, Sample, TimeSeriesSampleGenerator, TimeSeriesTask
from tsdm.utils.data import folds_as_frame, folds_as_sparse_frame, folds_from_groups
from tsdm.utils.types import KeyVar

In [ ]:
class KiwiSampleGenerator(TimeSeriesSampleGenerator):
    r"""Sample generator for the KIWI dataset."""

    def __init__(self, dataset):
        super().__init__(
            dataset,
            observables=[
                "Base",
                "DOT",
                "Glucose",
                "OD600",
                "Acetate",
                "Fluo_GFP",
                "pH",
            ],
            covariates=[
                "Cumulated_feed_volume_glucose",
                "Cumulated_feed_volume_medium",
                "InducerConcentration",
                "StirringSpeed",
                "Flow_Air",
                "Temperature",
                "Probe_Volume",
            ],
            targets=["Fluo_GFP"],
        )


class KiwiTask(TimeSeriesTask):
    r"""Task for the KIWI dataset."""
    # dataset: TimeSeriesCollection = KiwiDataset()
    observation_horizon: str = "2h"
    r"""The number of datapoints observed during prediction."""
    forecasting_horizon: str = "1h'"
    r"""The number of datapoints the model should forecast."""

    def __init__(self) -> None:
        super().__init__(dataset=KiwiDataset())

    @staticmethod
    def default_metric(*, targets, predictions):
        r"""TODO: implement this."""

    def default_collate(self):
        r"""TODO: implement this."""

    # def make_encoder(self, key: KeyVar, /) -> ModularEncoder:
    #     ...

    def make_sampler(self, key: KeyVar, /) -> TorchSampler:
        split: TimeSeriesCollection = self.splits[key]
        subsamplers = {
            key: SlidingWindowSampler(tsd.index, horizons=["2h", "1h"], stride="1h")
            for key, tsd in split.items()
        }
        return HierarchicalSampler(split, subsamplers, shuffle=False)  # type: ignore[return-value]

    def make_folds(self, /) -> DataFrame:
        r"""Group by RunID and color which indicates replicates."""
        md = self.dataset.metadata
        groups = md.groupby(["run_id", "color"], sort=False).ngroup()
        folds = folds_from_groups(
            groups, seed=2022, num_folds=5, train=7, valid=1, test=2
        )
        df = folds_as_frame(folds)
        return folds_as_sparse_frame(df)

    def make_generator(self, key: KeyVar, /) -> KiwiSampleGenerator:
        split = self.splits[key]
        return KiwiSampleGenerator(split)

In [ ]:
0 or False

In [ ]:
task = KiwiTask()

In [ ]:
task.dataset

In [ ]:
dataloader = task.dataloaders[(0, "train")]

In [ ]:
task.splits

In [ ]:
batch = next(iter(dataloader))

In [ ]:
sample = batch[0]

In [ ]:
sample.inputs

In [ ]:
raise

In [ ]:
sample.key

In [ ]:
sample._fields

In [ ]:
sample.inputs

## collate_fn without encoder

In [ ]:
def collate_fn(batch: list[Sample]) -> Batch:
    r"""Collate tensors into batch.

    Transform the data slightly: t, x, t_target → T, X where X[t_target:] = NAN
    """
    x_vals: list[Tensor] = []
    y_vals: list[Tensor] = []
    x_time: list[Tensor] = []
    y_time: list[Tensor] = []
    x_mask: list[Tensor] = []
    y_mask: list[Tensor] = []

    for sample in batch:
        t, x, t_target = sample.inputs
        y = sample.targets

        # get whole time interval
        time = torch.cat((t, t_target))
        sorted_idx = torch.argsort(time)

        # pad the x-values
        x_padding = torch.full(
            (t_target.shape[0], x.shape[-1]), fill_value=NAN, device=x.device
        )
        values = torch.cat((x, x_padding))

        # create a mask for looking up the target values
        mask_y = y.isfinite()
        mask_pad = torch.zeros_like(x, dtype=torch.bool)
        mask_x = torch.cat((mask_pad, mask_y))

        x_vals.append(values[sorted_idx])
        x_time.append(time[sorted_idx])
        x_mask.append(mask_x[sorted_idx])

        y_time.append(t_target)
        y_vals.append(y)
        y_mask.append(mask_y)

    return Batch(
        x_time=pad_sequence(x_time, batch_first=True).squeeze(),
        x_vals=pad_sequence(x_vals, batch_first=True, padding_value=NAN).squeeze(),
        x_mask=pad_sequence(x_mask, batch_first=True).squeeze(),
        y_time=pad_sequence(y_time, batch_first=True).squeeze(),
        y_vals=pad_sequence(y_vals, batch_first=True, padding_value=NAN).squeeze(),
        y_mask=pad_sequence(y_mask, batch_first=True).squeeze(),
    )

In [ ]:
collate_fn(batch)

## collate_fn using encoder!